# 🍷 Wine Quality Prediction using Multiple Linear Regression
## Goal: Predict wine quality (score 0–10) based on physicochemical features.


"""
Wine Quality Dataset used in this project is based on:

P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis (2009).
"Modeling wine preferences by data mining from physicochemical properties".
Decision Support Systems, 47(4):547-553.
https://doi.org/10.1016/j.dss.2009.05.016

Data obtained from the UCI Machine Learning Repository:
https://archive.ics.uci.edu/ml/datasets/wine+quality
"""


In [2]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from IPython.display import display

print("Importations complete.")

Importations complete.


## Load Data

In [3]:
df_wqw = pd.read_csv('winequality-white.csv', sep=';')
df_wqw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [4]:
df_wqr = pd.read_csv('winequality-red.csv', sep=';')
df_wqr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [9]:
df_wqw.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1231,7.9,0.410,0.37,4.50,0.030,40.0,114.0,0.99200,3.17,0.54,12.4,7
3566,7.6,0.380,0.28,4.20,0.029,7.0,112.0,0.99060,3.00,0.41,12.6,6
3625,7.7,0.320,0.61,11.80,0.041,66.0,188.0,0.99794,3.00,0.54,9.3,5
4716,5.5,0.315,0.38,2.60,0.033,10.0,69.0,0.99090,3.12,0.59,10.8,6
4198,7.9,0.360,0.53,12.90,0.049,63.0,139.0,0.99792,2.94,0.45,9.1,5
3024,7.0,0.130,0.37,12.85,0.042,36.0,105.0,0.99581,3.05,0.55,10.7,6
2631,6.7,0.280,0.28,2.40,0.012,36.0,100.0,0.99064,3.26,0.39,11.7,7
1537,8.1,0.280,0.49,1.00,0.040,32.0,148.0,0.99360,3.13,0.41,10.0,6
3214,5.2,0.335,0.20,1.70,0.033,17.0,74.0,0.99002,3.34,0.48,12.3,6
4691,6.9,0.190,0.31,19.25,0.043,38.0,167.0,0.99954,2.93,0.52,9.1,7


In [10]:
df_wqw.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000
mean,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267,5.877909
std,0.843868,0.100795,0.121020,5.072058,0.021848,17.007137,42.498065,0.002991,0.151001,0.114126,1.230621,0.885639
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991723,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000,6.000000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000,6.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000
